### Load libraries & config

In [ ]:
from datetime import date, datetime
import girder_client as gc
import json
import os
import pandas as pd
from urllib.parse import quote
import random
import re
import string
import tzlocal
from urllib.request import urlopen

with open("config.json", "r") as fp:
    config=json.load(fp)
    
def noResponseMessage(user, activity):
    """
    Parameters
    ----------
    user: dictionary
       firstName: string
       lastName: string
    
    activity: string
    
    Return
    ------
    responseMessage: string
    """
    return(
        "{} {}: No {} responses".format(
            user["firstName"],
            user["lastName"],
            activity
        )
    )

### Connect to Girder

In [ ]:
girder_connection = gc.GirderClient(
    apiUrl="{}/api/v1/".format(
        config["girder-production"]["host"]
    )
)
girder_connection.authenticate(
    username=config["girder-production"]["user"],
    password=config["girder-production"]["password"]
)

In [ ]:
activity_sets = girder_connection.get(
    "folder?parentType=collection&parentId={}".format(
        girder_connection.get(
            "collection?name=Volumes"
        )[0]["_id"]
    )
)
ema_parent_users = [
    activity_set for activity_set in activity_sets if
    activity_set['name']=="EMA: Parent"
][0]["meta"]["members"]["users"]

In [ ]:
response_table = {}
for user in ema_parent_users:
    user_id = girder_connection.getUser(user)
    response_table[user_id["_id"]] = {}
    try:
        responses = girder_connection.get(
            "folder?parentType=user&parentId={}&name={}".format(
                user,
                "Responses"
            )
        )
        if len(responses):
            for response_folder in responses:
                try:
                    ema_responses = girder_connection.get(
                        "folder?parentType=folder&parentId={}&name={}".format(
                            response_folder["_id"],
                            quote("EMA: Parent")
                        )
                    )
                    if len(ema_responses):
                        for ema_response_folder in ema_responses:
                            ema_response = girder_connection.get(
                                "item?folderId={}&limit=5000".format(
                                    ema_response_folder["_id"]
                                )
                            )
                            for response in ema_response:
                                if response[
                                    'meta'
                                ]['responseTime'] >= 1545368400000: # Mindlogger Healthy Brain Network beta test 1 start date
                                    if response[
                                        'meta'
                                    ]['activity']['name'] in response_table[
                                        user_id["_id"]
                                    ]:
                                        response_table[
                                            user_id["_id"]
                                        ][
                                            response['meta']['activity']['name']
                                        ] = response_table[
                                            user_id["_id"]
                                        ][
                                            response['meta']['activity']['name']
                                        ] + 1
                                    else:
                                        response_table[
                                            user_id["_id"]
                                        ][
                                            response['meta']['activity']['name']
                                        ] = 1
                    else:
                        print(noResponseMessage(user_id, "*EMA: Parent*"))
                except:
                    print(noResponseMessage(user_id, "*EMA: Parent*"))
        else:
            print(noResponseMessage(user_id, "Mindlogger"))
    except:
        print(noResponseMessage(user_id, "Mindlogger"))
pd.DataFrame(response_table).T.sum()